In [2]:
!dir

 Volume in drive C is System
 Volume Serial Number is 2CA8-E3EA

 Directory of C:\Users\amarthak\Documents\dg-spawn-chat-backend\opt\data

24-01-2020  17:38    <DIR>          .
24-01-2020  17:38    <DIR>          ..
24-01-2020  17:38    <DIR>          .ipynb_checkpoints
10-01-2020  10:00             6,967 spawn_en_data.json
10-01-2020  10:00             9,378 spawn_hi_data.json
24-01-2020  17:38                72 Untitled.ipynb
               3 File(s)         16,417 bytes
               3 Dir(s)  388,624,683,008 bytes free


In [6]:
import json
import json
import os
import pickle
import random
from multiprocessing.pool import ThreadPool
from pathlib import Path

import nltk
import numpy as np
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from nltk.stem.lancaster import LancasterStemmer

output_data = []
words_list = []
inputclasses = []
documents_vocab = []
train_xinput = []
train_youtput = []

Using TensorFlow backend.


In [7]:
with open('spawn_en_data.json', encoding='UTF-8') as f:
        data = json.load(f)

In [11]:
output_data = list(data.get('intents'))
print("%s sentences in training data" % len(output_data))
#print(output_data)
print("%s sentences in training data" % len(output_data))

ignore_words = ['?']
for pattern in output_data:
    w = nltk.word_tokenize(pattern['text'])
    words_list.extend(w)
    documents_vocab.append((w, pattern['intent']))
    if pattern['intent'] not in inputclasses:
        inputclasses.append(pattern['intent'])

90 sentences in training data
90 sentences in training data


In [13]:
stemmer = LancasterStemmer()
words_list = [stemmer.stem(w.lower())
                  for w in words_list if w not in ignore_words]

In [14]:
words_list = sorted(list(set(words_list)))

In [16]:
words_list

["'s",
 'a',
 'about',
 'afternoon',
 'am',
 'an',
 'anyth',
 'ar',
 'awesom',
 'bitch',
 'bye',
 'bye-by',
 'can',
 'catch',
 'do',
 'doing',
 'dont',
 'dumb',
 'fantast',
 'fin',
 'fuck',
 'good',
 'goodby',
 'gre',
 'hav',
 'hello',
 'help',
 'hey',
 'hi',
 'how',
 'i',
 'id',
 'idiot',
 'inspir',
 'keep',
 'know',
 'lastest',
 'lat',
 'latest',
 'lik',
 'lov',
 'me',
 'morn',
 'mot',
 'motherfuck',
 'mou',
 "n't",
 'nam',
 'nee',
 'new',
 'no',
 'not',
 'noth',
 'ok',
 'profess',
 'quiet',
 'quot',
 'see',
 'send',
 'show',
 'shut',
 'skil',
 'spawn',
 'stupid',
 'talk',
 'tel',
 'thank',
 'the',
 'ther',
 'thing',
 'to',
 'today',
 'up',
 'want',
 'what',
 'who',
 'yo',
 'you']

In [17]:
inputclasses = sorted(list(set(inputclasses)))

In [18]:
inputclasses

['afernoon',
 'bot_bye',
 'bot_greet_reply',
 'bot_hi',
 'bot_identity',
 'bot_love',
 'bot_negative',
 'bot_news',
 'bot_quotes',
 'bot_shutup',
 'bot_skills',
 'bot_thanks',
 'greet',
 'morning',
 'spawn_name']

In [19]:
data = {}
data['words'] = words_list
data['classes'] = inputclasses

In [20]:
data

{'words': ["'s",
  'a',
  'about',
  'afternoon',
  'am',
  'an',
  'anyth',
  'ar',
  'awesom',
  'bitch',
  'bye',
  'bye-by',
  'can',
  'catch',
  'do',
  'doing',
  'dont',
  'dumb',
  'fantast',
  'fin',
  'fuck',
  'good',
  'goodby',
  'gre',
  'hav',
  'hello',
  'help',
  'hey',
  'hi',
  'how',
  'i',
  'id',
  'idiot',
  'inspir',
  'keep',
  'know',
  'lastest',
  'lat',
  'latest',
  'lik',
  'lov',
  'me',
  'morn',
  'mot',
  'motherfuck',
  'mou',
  "n't",
  'nam',
  'nee',
  'new',
  'no',
  'not',
  'noth',
  'ok',
  'profess',
  'quiet',
  'quot',
  'see',
  'send',
  'show',
  'shut',
  'skil',
  'spawn',
  'stupid',
  'talk',
  'tel',
  'thank',
  'the',
  'ther',
  'thing',
  'to',
  'today',
  'up',
  'want',
  'what',
  'who',
  'yo',
  'you'],
 'classes': ['afernoon',
  'bot_bye',
  'bot_greet_reply',
  'bot_hi',
  'bot_identity',
  'bot_love',
  'bot_negative',
  'bot_news',
  'bot_quotes',
  'bot_shutup',
  'bot_skills',
  'bot_thanks',
  'greet',
  'morning

In [21]:
with open('data.json', 'w') as f:
    json.dump(data, f)

In [22]:
training = []
output_data = []
output_empty = [0] * len(inputclasses)

for doc in documents_vocab:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words_list:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[inputclasses.index(doc[1])] = 1

    training.append([bag, output_row])

In [23]:
random.shuffle(training)
training = np.array(training)
train_xinput = list(training[:, 0])
train_youtput = list(training[:, 1])

In [24]:
model_nn = Sequential()
model_nn.add(Dense(32, input_dim=len(train_xinput[0]), activation='relu'))
model_nn.add(Dense(16, activation='relu'))
model_nn.add(Dense(len(train_youtput[0]), activation='softmax'))

model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nn.fit(np.array(train_xinput), np.array(train_youtput), epochs=95, batch_size=8)

model_nn.save('spawn_en.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/95
180/180 [==============================] - 1s 5ms/step - loss: 2.6862 - accuracy: 0.0833
Epoch 2/95
180/180 [==============================] - 0s 127us/step - loss: 2.6274 - accuracy: 0.3222
Epoch 3/95
180/180 [==============================] - 0s 128us/step - loss: 2.5604 - accuracy: 0.3667
Epoch 4/95
180/180 [==============================] - 0s 122us/step - loss: 2.4788 - accuracy: 0.3667
Epoch 5/95
180/180 [==============================] - 0s 122us/step - loss: 2.3829 - accuracy: 0.4333
Epoch 6/95
180/180 [==============================] - 0s 122us/step - loss: 2.2667 - accuracy: 0.4556
Epoch 7/95
180/180 [==============================] - 0s 111us/step - loss: 2.1285 - accuracy: 0.4889
Epoch 8/95
180/180 [==============================] - 0s 127us/step - loss: 1.9775 - accuracy: 0.5000
Epoch 9/95
180/180 [==============================] - 0s 116us/step - loss: 1.8183 - accuracy: 0.5278
Ep

180/180 [==============================] - 0s 94us/step - loss: 0.0240 - accuracy: 0.9889
Epoch 76/95
180/180 [==============================] - 0s 100us/step - loss: 0.0245 - accuracy: 0.9889
Epoch 77/95
180/180 [==============================] - 0s 100us/step - loss: 0.0237 - accuracy: 0.9889
Epoch 78/95
180/180 [==============================] - 0s 100us/step - loss: 0.0241 - accuracy: 0.9778
Epoch 79/95
180/180 [==============================] - 0s 94us/step - loss: 0.0233 - accuracy: 0.9889
Epoch 80/95
180/180 [==============================] - 0s 94us/step - loss: 0.0237 - accuracy: 0.9889
Epoch 81/95
180/180 [==============================] - 0s 94us/step - loss: 0.0235 - accuracy: 0.9778
Epoch 82/95
180/180 [==============================] - 0s 100us/step - loss: 0.0225 - accuracy: 0.9889
Epoch 83/95
180/180 [==============================] - 0s 94us/step - loss: 0.0238 - accuracy: 0.9778
Epoch 84/95
180/180 [==============================] - 0s 100us/step - loss: 0.0239 - accu

In [ ]:
!tflite_convert --keras_model_file=spawn_en.h5 --output_file=spawn_en.tflite